In [ ]:
# coding=utf8


from numpy import mean, absolute
import os
import MyQueue
import numpy as np
import scipy
import random
import fractions

In [ ]:
#write a file 
def writecsv(addlabel,filename):
    with open(filename, 'a') as the_file:
        for i in addlabel:
            the_file.write(str(i)[1:-1].replace('\'','')+'\n')

In [ ]:
# Set the time window to 1 second, read the csv files in the addlabel file.
# Each file within the same second is considered as a window. Remove the first and last two time windows, reduce the sampling rate within each window, and store the data with reduced sampling rate in the reduce file. 
# this is for user observation. Extract features from the data within each window to generate an arff file.
def window_slide(filepath,sample_rate):
     featurelist=[]
     for root, dirs, files in os.walk(filepath):
        for fn in files:
            # if 'HASC22203' in fn:
                filename = root+'/'+fn
                # Read a CSV file.
                FileID= filename.split('/')[-1].split('.')[0]
                PersonID = filename.split('/')[-2][0:10]
                sampleRate=0
                if sample_rate!=25:
                    sampleRate=sample_rate*5
                else:
                    sampleRate=4
                with  open(filename, 'r') as f:
                    lines=f.readlines()
                    linelist = []
                    # Each file is placed in a time window based on the position of the decimal point before the time value. The time window is set to 2 seconds.
                    for line in lines:
                        time,x,y,z,label= line.strip().split(',')
                        if len(linelist)==0:
                            s = MyQueue.Queue()
                            t = (time,x,y,z,label,FileID,PersonID,sampleRate)
                            s.enqueue(t)
                            linelist.append(s)

                        else:
                            s = linelist[-1]
                            pretime=s.peek()[0][0:10]
                            if time[0:10]==pretime :#or int(time[0:10])==int(pretime)+1 If the time is the same or the interval is one second, they are considered as belonging to the same window.
                                t = (time,x,y,z,label,FileID,PersonID,sampleRate)
                                s.enqueue(t)
                            else:
                                s = MyQueue.Queue()
                                t = (time,x,y,z,label,FileID,PersonID,sampleRate)
                                s.enqueue(t)
                                linelist.append(s)

                    linelist.pop(0)
                    linelist.pop()
                #
                    reducelist=[]
                    for index,value in enumerate(linelist):
                        #  the sampling rate, with values of 1, 2, 4, 5, 10, 20, or 25. The original data is 200 Hz, meaning there are 200 samples per second. 
                        #  If "sample_rate" is set to 2, then the sampling rate is reduced to half, resulting in 100 samples per second.
                        if sampleRate==4 :
                            w=value.elements[0:len(value.elements):sample_rate]
                            reducelist.append(w)
                        elif sampleRate==100 :
                                w=value.elements[0:len(value.elements):1]
                                reducelist.append(w)
                        else:
                            tmp=[]
                            for i in range(0,len(value.elements),20):
                                b=value.elements[i:i+20]
                                tmp+=b[0:sample_rate]
                            reducelist.append(tmp)

                    # reduceoutput=[]
                    # for i in reducelist:
                    #     for j in  i:
                    #         reduceoutput.append((j[0],j[1],j[2],j[3]))
                    # writecsv(reduceoutput,'reduce5/'+fn)


                    count=0
                    for i,v in  enumerate(reducelist):
                        if len(v)>=4:
                            fs=getFeature(v)
                            count+=1
                            oneline = fs+(count,)
                            featurelist.append(oneline)


     return featurelist

In [ ]:
def meanad(data, axis=None):
    return mean(absolute(data - mean(data, axis)), axis)

def percentile_70(axis):
    return  np.percentile(axis, 70)


def percentile_80(axis):
    return  np.percentile(axis, 80)


def percentile_90(axis):
    return  np.percentile(axis, 90)

SAMPLES_PER_SECOND = 100
def peak_frequency(axis, SAMPLES_PER_SECOND):
    spacing=1.0 / SAMPLES_PER_SECOND
    spectrum = scipy.fft(axis)[:len(axis) / 2]
    freqs = np.fft.fftfreq(len(spectrum), d=spacing)[:len(axis) / 2]
    spectrum[0] = 0
    return freqs[np.argmax(np.abs(spectrum))]

def energy(axis, spacing=1.0 / SAMPLES_PER_SECOND):
    spectrum = scipy.fft(axis)[:len(axis) / 2]
    return np.sqrt(np.sum(np.square(np.abs(spectrum))) / len(axis))

In [ ]:
def getFeature(windowlist):
    time=[]
    x=[]
    y=[]
    z=[]
    UnixTime=windowlist[0][0][0:10]
    label=windowlist[0][4]
    FileID=windowlist[0][5]
    PersonID=windowlist[0][6]
    sample_rate=windowlist[0][7]
    for element in windowlist :
        time.append(element[0])
        x.append(float(element[1]))
        y.append(float(element[2]))
        z.append(float(element[3]))
    # print x
    NUMBER_FORMAT = "%.6f"
    x=np.array(x)
    y=np.array(y)
    z=np.array(z)
    x70=NUMBER_FORMAT % percentile_70(x)
    y70=NUMBER_FORMAT % percentile_70(y)
    z70=NUMBER_FORMAT % percentile_70(z)
    x80=NUMBER_FORMAT % percentile_80(x)
    y80=NUMBER_FORMAT % percentile_80(y)
    z80=NUMBER_FORMAT % percentile_80(z)
    x90=NUMBER_FORMAT % percentile_90(x)
    y90=NUMBER_FORMAT % percentile_90(y)
    z90=NUMBER_FORMAT % percentile_90(z)
    px=NUMBER_FORMAT % peak_frequency(x,sample_rate)
    py=NUMBER_FORMAT % peak_frequency(y,sample_rate)
    pz=NUMBER_FORMAT % peak_frequency(z,sample_rate)
    energyx=NUMBER_FORMAT % energy(x)
    energyy=NUMBER_FORMAT % energy(y)
    energyz=NUMBER_FORMAT % energy(z)
    sx = NUMBER_FORMAT % np.std(x)
    sy = NUMBER_FORMAT % np.std(y)
    sz = NUMBER_FORMAT % np.std(z)
    averagex = NUMBER_FORMAT % np.average(x)
    averagey = NUMBER_FORMAT % np.average(y)
    averagez = NUMBER_FORMAT % np.average(z)
    medianx = NUMBER_FORMAT % np.median(x)
    mediany = NUMBER_FORMAT % np.median(y)
    medianz = NUMBER_FORMAT % np.median(z)
    minx = NUMBER_FORMAT % np.nanmin(x)
    miny = NUMBER_FORMAT % np.nanmin(y)
    minz = NUMBER_FORMAT % np.nanmin(z)
    varx = NUMBER_FORMAT % np.var(x)
    vary = NUMBER_FORMAT % np.var(y)
    varz = NUMBER_FORMAT % np.var(z)
    maxx = NUMBER_FORMAT % np.nanmax(x)
    maxy = NUMBER_FORMAT % np.nanmax(y)
    maxz = NUMBER_FORMAT % np.nanmax(z)
    madx = NUMBER_FORMAT % meanad(x)
    mady = NUMBER_FORMAT % meanad(y)
    madz = NUMBER_FORMAT % meanad(z)
    return PersonID,label,FileID,UnixTime,sample_rate,energyx,energyy,energyz,x70,y70,z70,sx,sy,sz,px,py,pz,madx,mady,madz,averagex,averagey,averagez,\
           medianx,mediany,medianz,x80,y80,z80,x90,y90,z90,minx,miny,minz,\
           varx,vary,varz,maxx,maxy,maxz

In [ ]:
SENSORDIRS = {'accelerometer': r"HASC1001-acc.csv"}
AXES = (('x', 3), ('y', 2), ('z', 1))
#the features we use
SINGLE_FEATURES = {
    'average-absolute-difference': meanad,'standarddev': np.std, 'variance': np.var,
    'nanmin':np.nanmin, 'nanmax':np.nanmax, 'median':np.median, 'average':np.average,
    'percentile_70':percentile_70,'percentile_80':percentile_80,'percentile_90':percentile_90,
    'dominant-frequency': peak_frequency,'energy': energy
} #all features working on one axis


In [ ]:
def main():
    path='/personall_old'

    num=[]
    for i in range(1,21):
        num.append(i)
    num.append(25)
    for i in num:
        chooselist= []
        for root, dirs, files in os.walk(path):
            if 'Person' in root:
                person=int(root[57:61])
                # person=int(root[60:64])
                features = window_slide(root,i)
                chooselist.append(features)
    #
        with open('/AllFeature3.csv', 'a') as the_file:
                for c in chooselist:
                    for i in  c:
                        if '@' not in i:
                            the_file.write(str(i)[1:-1].replace('\'','')+'\n')
                        else:
                            the_file.write(i+'\n')

In [ ]:
if __name__ == '__main__':
    main()